## Extract pixel values in Google Earth Engine (GEE) for Fluxnet sites

In [ ]:
!pip install earthengine-api

For more information about the installation of Google Earth Engine (Python Api) , please visit https://developers.google.com/earth-engine/guides/python_install

In [ ]:
import os
import ee

Authenticate to the Earth Engine servers:

In [ ]:
ee.Authenticate()

Initialize the API:

In [ ]:
ee.Initialize()

In [ ]:
def func_feat(feature,index):
    """ Orgnazition of output csv

    """
    coordinates = feature.geometry().coordinates()
    feature = feature.set("lon", coordinates.get(0))
    feature = feature.set("lat", coordinates.get(1))
    feature = feature.set("imageId", index)
    feature = feature.set("pointId", feature.get("system:index"))
    return feature

def func_cov(image,points):
    """ get mean value in the area defined by scale

    """
    scale = 500
    fCol = image.reduceRegions(collection = points,
                  reducer = ee.Reducer.mean(),
                  scale = scale)
    index= image.get("system:index")
    fCol = fCol.map(lambda x: func_feat(x,index))
    return fCol

You need to upload the [locations of Fluxnet sites](https://fluxnet.org/sites/site-list-and-pages/) in shape files format to your GEE assets **or** open this [link](https://code.earthengine.google.com/?asset=users/mrhuaize/fluxnet_sites) to copy it to your assets directly. 

In [ ]:
dataset = "MODIS/006/MCD43A4" #check details and more products on Earth Engine Data Catalog
loc_of_fluxnet_towers = 'users/mrhuaize/fluxnet_sites' 
start_time = "2017-1-1"
end_time = "2017-1-5"# Notice that GEE up to 5000 records can be extracted at on time
band = "Nadir_Reflectance_Band1" # check details especially the scale factor on Earth Engine Data Catalog

In [ ]:
l8 = ee.ImageCollection(dataset)
points = ee.FeatureCollection(loc_of_fluxnet_towers)
scol = l8.filterDate(start_time,end_time).select(band)
scol = scol.map(lambda img: func_cov(img,points)).flatten()

In [ ]:
# Export the table to your Google Drive 
task = ee.batch.Export.table.toDrive(
  collection = scol,
  description = "fluxnet_"+band,
  fileFormat = "CSV",
  selectors =  ["imageId", "pointId", "lon", "lat", "mean"]
)
task.start()

In [20]:
import pandas as pd
from datetime import datetime
# Post-processing if you intrest
def time2Index(x):
  """ Dealing with timestamp

  """
  t = []
  ts  = x.astype(np.str).values
  for time in ts:
      t.append(datetime.strptime(time,'%Y_%m_%d'))
  return t

def getGEEvar(file,vari,scale,time = '30min'):
  """ Scaling and interpolate

  """
  LAI = pd.read_csv(file)
  LAI.imageId = pd.DatetimeIndex(time2Index(LAI.imageId)).round('min')
  LAI.columns = LAI.columns.str.replace('imageId','TIMESTAMP')
  LAI.columns = LAI.columns.str.replace('mean',vari)
  LAI.set_index('TIMESTAMP',inplace = True)
  LAI = LAI.resample(time).interpolate()
  LAI = LAI.drop(['lon','lat','pointId'],axis = 1)#.resample(time).interpolate()
  LAI[vari] = LAI[vari]*scale
  return LAI